# Dynamic Treatment Regime

In [73]:
import numpy as np
import scipy.special

# generate dynamic data
def gen_data(alpha, A, B, c, n, d, tau1=None, tau2=None):
    S1 = np.random.normal(0, 1, size=(n, d))
    if tau1 is None:
        D1 = np.random.binomial(1, scipy.special.expit(S1 @ alpha))
    else:
        D1 = tau1 * np.ones(n)
    S2 = S1 @ A.T + D1.reshape(-1, 1) @ B + np.random.normal(0, 1, size=(n, d))
    if tau2 is None:
        D2 = np.random.binomial(1, scipy.special.expit(S2 @ alpha))
    else:
        D2 = tau2 * np.ones(n)
    Y = D2 + S2 @ c + np.random.normal(0, 1, size=(n,))
    return S1, D1, S2, D2, Y

In [74]:
# instance parameters
d = 4
alpha = np.random.uniform(-1, 1, size=d)
A = np.random.uniform(-1, 1, size=(d, d))
B = np.random.uniform(-1, 1, size=(1, d))
c = np.random.uniform(-1, 1, size=d)

In [82]:
# observational treatment regime mean outcome
n = 1000000
np.mean(gen_data(alpha, A, B, c, n, d)[-1])

0.29795130536548675

In [92]:
# expected value of counterfactual treatment regime (0, 0)
n = 1000000
V00 = np.mean(gen_data(alpha, A, B, c, n, d, 0, 0)[-1])
V00

0.0009090676173456719

In [93]:
# expected value of counterfactual treatment regime (1, 1)
n = 1000000
V11 = np.mean(gen_data(alpha, A, B, c, n, d, 1, 1)[-1])
V11

0.6068395938150491

# Generate Observed Data

In [85]:
n = 5000
S1, D1, S2, D2, y = gen_data(alpha, A, B, c, n, d)

# Wrong Identification by Conditioning of Effects with OLS

In [97]:
from sklearn.linear_model import LinearRegression, LogisticRegression

effects = LinearRegression().fit(np.column_stack([D1, D2, S1, S2]), y).coef_[:2]

In [98]:
# estimated expected value of counterfactual treatment regime (0, 0)
V00hat = np.mean(y - effects[0] * D1 - effects[1] * D2)
print(f'Estimate: {V00hat:.4f}, True: {V00:.4f}')

Estimate: -0.2105, True: 0.0009


In [99]:
# estimated expected value of counterfactual treatment regime (1, 1)
V11hat = np.mean(y - effects[0] * (D1 - 1) - effects[1] * (D2 - 1))
print(f'Estimate: {V11hat:.4f}, True: {V11:.4f}')

Estimate: 0.7557, True: 0.6068


# G-Computation

In [108]:
from sklearn.linear_model import LinearRegression, LogisticRegression

def cntf(S1, D1, S2, D2, y, tau1, tau2, modely1, modely2):
    n = S1.shape[0]
    my2 = modely2.fit(np.column_stack([D2, S2]), y)
    Yadj = my2.predict(np.column_stack([tau2 * np.ones(n), S2]))

    my1 = modely1.fit(np.column_stack([D1, S1]), Yadj)
    Yadj = my1.predict(np.column_stack([tau1 * np.ones(n), S1]))

    return np.mean(Yadj)

short_cntf = lambda tau1, tau2: cntf(S1, D1, S2, D2, y, tau1, tau2, LinearRegression(), LinearRegression())

In [109]:
print(f'Estimate: {short_cntf(0, 0):.4f}, True: {V00:.4f}')

Estimate: -0.0213, True: 0.0009


In [110]:
print(f'Estimate: {short_cntf(1, 1):.4f}, True: {V11:.4f}')

Estimate: 0.5602, True: 0.6068


# Doubly (Multiply) Robust Estimation and Confidence Interval

In [111]:
from sklearn.linear_model import LinearRegression, LogisticRegression

def drcntf(S1, D1, S2, D2, y, tau1, tau2, modely1, modely2, modeld1, modeld2):
    n = S1.shape[0]
    
    my2 = modely2.fit(np.column_stack([D2, S2]), y)
    resy2 = y - my2.predict(np.column_stack([D2, S2]))
    Yadj2 = my2.predict(np.column_stack([tau2 * np.ones(n), S2]))
    
    md2 = modeld2.fit(S2, D2)
    prop2 = md2.predict_proba(S2)[:, 1]
    ips2 = tau2 * (D2 == 1) / prop2 + (1 - tau2) * (D2 == 0) / (1 - prop2)

    my1 = modely1.fit(np.column_stack([D1, S1]), Yadj2)
    resy1 = Yadj2 - my1.predict(np.column_stack([D1, S1]))
    Yadj1 = my1.predict(np.column_stack([tau1 * np.ones(n), S1]))

    md1 = modeld1.fit(S1, D1)
    prop1 = md1.predict_proba(S1)[:, 1]
    ips1 = tau1 * (D1 == 1) / prop1 + (1 - tau1) * (D1 == 0) / (1 - prop1)

    dr = Yadj1 + ips1 * resy1 + ips1 * ips2 * resy2
    return np.mean(dr), np.std(dr) / np.sqrt(n)

short_drcntf = lambda tau1, tau2: drcntf(S1, D1, S2, D2, y, tau1, tau2,
                                         LinearRegression(), LinearRegression(),
                                         LogisticRegression(C=1000), LogisticRegression(C=1000))

In [112]:
V00hat, V00se = short_drcntf(0, 0)
print(f'Estimate: {V00hat:.4f} ({V00se:.4f}), True: {V00:.4f}')

Estimate: -0.0266 (0.0586), True: 0.0009


In [113]:
V11hat, V11se = short_drcntf(1, 1)
print(f'Estimate: {V11hat:.4f} ({V11se:.4f}), True: {V11:.4f}')

Estimate: 0.5456 (0.0437), True: 0.6068


# G-Estimation: Dynamic Partialling Out (Neyman Orthogonal)

In [126]:
from sklearn.linear_model import LinearRegression, LogisticRegression
import scipy.linalg

def dynplr(S1, D1, S2, D2, y, tau1, tau2, modely1, modely2, modeld1, modeld2):
    n = S1.shape[0]

    psi = np.zeros((n, 3))
    J = np.zeros((3, 3))
    
    # Estimate second period treatment effect using Partialling Out
    my2 = modely2.fit(S2, y)
    resy2 = y - my2.predict(S2)
    md2 = modeld2.fit(S2, D2)
    resD2 = D2 - md2.predict_proba(S2)[:, 1]
    delta2 = np.mean(resy2 * resD2) / np.mean(resD2**2) # effect of D2
    
    # moment function for delta2
    psi[:, 2] = (resy2 - delta2 * resD2) * resD2
    # jacobian of moment
    J[2, 2] = - np.mean(resD2**2)

    # Adjust the outcome for second period treatment
    Yadj2 = y - delta2 * (D2 - tau2)

    # Estimate first period treatment effect using Partialling Out
    my1 = modely1.fit(S1, Yadj2)
    resy1 = Yadj2 - my1.predict(S1)
    md1 = modeld1.fit(S1, D1)
    resD1 = D1 - md1.predict_proba(S1)[:, 1]
    delta1 = np.mean(resy1 * resD1) / np.mean(resD1**2) # effect of D1
    
    # moment function for delta1
    psi[:, 1] = (resy1 - delta1 * resD1) * resD1
    # jacobian of moment
    J[1, 1:] = - np.array([np.mean(resD1**2), np.mean(resD1 * (D1 - tau1))])
    
    # Adjust the outcome for first period treatment
    Yadj1 = Yadj2 - delta1 * (D1 - tau1)
    
    # Mean counterfactual value estimate
    theta = np.mean(Yadj1)
    
    # moment for theta
    psi[:, 0] = Yadj1 - theta
    # jacobian of moment
    J[0, :] = - np.array([1, np.mean(D1 - tau1), np.mean(D2 - tau2)])
    
    # calculating covariance of (theta, delta1, delta2)
    Jinv = scipy.linalg.pinv(J)
    Sigma = psi.T @ psi / n
    V = Jinv @ Sigma @ Jinv.T

    return theta, np.sqrt(V[0, 0] / n)

short_dynplr = lambda tau1, tau2: dynplr(S1, D1, S2, D2, y, tau1, tau2,
                                         LinearRegression(), LinearRegression(),
                                         LogisticRegression(C=1000), LogisticRegression(C=1000))

In [127]:
V00hat, V00se = short_dynplr(0, 0)
print(f'Estimate: {V00hat:.4f} ({V00se:.4f}), True: {V00:.4f}')

Estimate: -0.0084 (0.0310), True: 0.0009


In [128]:
V11hat, V11se = short_dynplr(1, 1)
print(f'Estimate: {V11hat:.4f} ({V11se:.4f}), True: {V11:.4f}')

Estimate: 0.5469 (0.0320), True: 0.6068
